In [2]:
!conda install -c conda-forge folium=0.5.0 --yes

import pandas as pd
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be 

In [24]:
# define Foursquare Credentials and Version
CLIENT_ID = 'CLIENT_ID' # your Foursquare ID
CLIENT_SECRET = 'CLIENT_SECRET' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CLIENT_ID
CLIENT_SECRET:CLIENT_SECRET


First a GET request for FourSquare API is constructed. Here, we find coffee shop near Singapore center, in 1000 m radius. In general, it's better to let requests to handle adding parameters to url than manually construct it.

In [4]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": VERSION,
    "section": "coffee",
    "near": "Singapore",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [6]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

Most of the items on dictionary are giving some general information like the name of the city, the related geo code and etc.  Groups contain the actual search result.

In [7]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Singapore', 'Singapore')

In [8]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 1.299030242682696, 'lng': 103.85714744704715},
  'sw': {'lat': 1.2803469036633846, 'lng': 103.84187725680208}},
 131)

In [9]:
d["geocode"]

{'what': '',
 'where': 'singapore',
 'center': {'lat': 1.28967, 'lng': 103.85007},
 'displayString': 'Singapore, Singapore',
 'cc': 'SG',
 'geometry': {'bounds': {'ne': {'lat': 1.4708569296084157,
    'lng': 104.03492993227736},
   'sw': {'lat': 1.2376256937422347, 'lng': 103.61698784239873}}},
 'slug': 'singapore',
 'longId': '72057594039808188'}

Let's print couple of first item to get better idea of the data structure.

In [10]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [11]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [12]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 50


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5d170ae66d54f8002357e288',
  'name': 'PPP Coffee',
  'location': {'address': '#02-19 Funan',
   'crossStreet': '107 North Bridge Road',
   'lat': 1.291628,
   'lng': 103.849741,
   'labeledLatLngs': [{'label': 'display',
     'lat': 1.291628,
     'lng': 103.849741}],
   'postalCode': '179095',
   'cc': 'SG',
   'neighborhood': 'Downtown Core',
   'city': 'Singapore',
   'country': 'Singapore',
   'formattedAddress': ['#02-19 Funan (107 North Bridge Road)',
    '179095',
    'Singapore']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-5d170ae6

In [13]:
items[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '52fddabd498e48893061a262',
  'name': 'Starbucks Reserve Store (Starbucks Reserve)',
  'location': {'address': '#02-01/02/03 The Fullerton Waterboat House',
   'crossStreet': '3 Fullerton Road',
   'lat': 1.2870494697229158,
   'lng': 103.85359982710769,
   'labeledLatLngs': [{'label': 'display',
     'lat': 1.2870494697229158,
     'lng': 103.85359982710769}],
   'postalCode': '049215',
   'cc': 'SG',
   'city': 'Singapore',
   'country': 'Singapore',
   'formattedAddress': ['#02-01/02/03 The Fullerton Waterboat House (3 Fullerton Road)',
    '049215',
    'Singapore']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'prim

In [20]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address = location["formattedAddress"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address","postalcode", "lat", "lng"])
print("found %i coffee" % len(df))
df.head()

found 39 coffee


,uid,name,shortname,address,postalcode,lat,lng
0,5d170ae66d54f8002357e288,PPP Coffee,Coffee Shop,"[#02-19 Funan (107 North Bridge Road), 179095,...",179095,1.291628,103.849741
1,52fddabd498e48893061a262,Starbucks Reserve Store (Starbucks Reserve),Coffee Shop,[#02-01/02/03 The Fullerton Waterboat House (3...,049215,1.287049,103.853600
2,527ba4d1498e06c5aa18aaca,RONIN,Café,"[17 Hongkong St, 059660, Singapore]",059660,1.287708,103.847177
3,55880c7b498eef1e8810c90d,Hoshino Coffee,Café,"[#B2-55 Capitol Piazza (13 Stamford Road), 178...",178905,1.292988,103.851441
4,5b348bd80d8a0f002cb3f0ed,The Stamford Brasserie,Café,"[2 Stamford Rd, 178882, Singapore]",178882,1.292836,103.853693


Some density based on estimator is giving a good tip where to start a new coffee shop.  There is a HeatMap to visualize all the existing coffee shops to same map.

In [21]:
singapore_center = d["geocode"]["center"]
singapore_center

{'lat': 1.28967, 'lng': 103.85007}

In [22]:
from folium import plugins

# create map of Singapore using latitude and longitude values
map_singapore = folium.Map(location=[singapore_center["lat"], singapore_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_singapore)

add_markers(df)
hm_data = df[["lat", "lng"]].as_matrix().tolist()
map_singapore.add_child(plugins.HeatMap(hm_data))

map_singapore

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


## Results

Based on these results, one possibly good location for the new Coffee Shop would be in crossroad of North Bridge Road and High Street, which is inside the park near the Parliament of Singapore.

In [23]:
lat = 1.28967
lng = 103.85007
map_singapore = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Our Cafe!",
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_singapore)
map_singapore